In [1]:
from keras.models import Sequential
from sklearn.model_selection import train_test_split

import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision import datasets
import glob

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device: torch.device = torch.device("cpu"),


In [2]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [3]:
data_dir = "./data"
categories = []

In [4]:
for i in range(1, 101):
    categories.append(f"{i}")

In [5]:
print(categories)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']


In [6]:
from PIL import Image

num_classes = len(categories)

image_w = 224
image_h = 224

pixels = image_w * image_h * 3

x = []
y = []

for idx, category in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idx] = 1

    image_dir = data_dir + "/" + category
    files = glob.glob(image_dir + "/*.jpg")

    for i,f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        x.append(data)
        y.append(label)

        if i % 700 == 0:
            print(category, " : ", f)

1  :  ./data/1\1_0_0_20161219140623097.jpg.chip.jpg
1  :  ./data/1\1_1_0_20170109190936410.jpg.chip.jpg
2  :  ./data/2\2_0_0_20161219154008997.jpg.chip.jpg
3  :  ./data/3\3_0_0_20161219154520213.jpg.chip.jpg
4  :  ./data/4\4_0_0_20161219192808843.jpg.chip.jpg
5  :  ./data/5\5_0_0_20170103205053570.jpg.chip.jpg
6  :  ./data/6\6_0_0_20170110213109002.jpg.chip.jpg
7  :  ./data/7\7_0_0_20161219201514284.jpg.chip.jpg
8  :  ./data/8\8_0_0_20170103200436055.jpg.chip.jpg
9  :  ./data/9\9_0_0_20170102235106821.jpg.chip.jpg
10  :  ./data/10\10_0_0_20161220222308131.jpg.chip.jpg
11  :  ./data/11\11_0_0_20170103200509559.jpg.chip.jpg
12  :  ./data/12\12_0_0_20170103200900511.jpg.chip.jpg
13  :  ./data/13\13_0_0_20170103200413990.jpg.chip.jpg
14  :  ./data/14\14_0_0_20170102234323550.jpg.chip.jpg
15  :  ./data/15\15_0_0_20170102234355667.jpg.chip.jpg
16  :  ./data/16\16_0_0_20170102234641453.jpg.chip.jpg
17  :  ./data/17\17_0_0_20170103201439825.jpg.chip.jpg
18  :  ./data/18\18_0_0_2017010320130800

In [7]:
x = np.array(x)
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
xy = (x_train, x_test, y_train, y_test)

x_train = x_train.astype(float) / 255
x_test = x_test.astype(float) / 255

ok 23685


In [23]:
from keras.layers import Input, Conv2D, Activation, MaxPool2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.models import Model

input = Input(shape=(224, 224, 3))

cnn1 = Conv2D(128, kernel_size=3, activation='relu')(input)
cnn1 = Conv2D(128, kernel_size=3, activation='relu')(cnn1)
cnn1 = Conv2D(128, kernel_size=3, activation='relu')(cnn1)
cnn1 = MaxPool2D(pool_size=3, strides=2)(cnn1)

cnn2 = Conv2D(128, kernel_size=3, activation='relu')(cnn1)
cnn2 = Conv2D(128, kernel_size=3, activation='relu')(cnn2)
cnn2 = Conv2D(128, kernel_size=3, activation='relu')(cnn2)
cnn2 = MaxPool2D(pool_size=3, strides=2)(cnn2)

cnn3 = Conv2D(256, kernel_size=3, activation='relu')(cnn2)
cnn3 = Conv2D(256, kernel_size=3, activation='relu')(cnn3)
cnn3 = Conv2D(256, kernel_size=3, activation='relu')(cnn3)
cnn3 = MaxPool2D(pool_size=3, strides=2)(cnn3)

cnn4 = Conv2D(512, kernel_size=3, activation='relu')(cnn3)
cnn4 = Conv2D(512, kernel_size=3, activation='relu')(cnn4)
cnn4 = Conv2D(512, kernel_size=3, activation='relu')(cnn4)
cnn4 = MaxPool2D(pool_size=3, strides=2)(cnn4)

dense = Flatten()(cnn4)
dense = Dropout(0.2)(dense)
dense = Dense(1024, activation='relu')(dense)
dense = Dense(1024, activation='relu')(dense)

output = Dense(1, activation='linear', name='age')(dense)

model = Model(input, output)
model.compile(optimizer=Adam(0.0001), loss='mse', metrics=['mae'])

In [24]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 222, 222, 128)     3584      
                                                                 
 conv2d_13 (Conv2D)          (None, 220, 220, 128)     147584    
                                                                 
 conv2d_14 (Conv2D)          (None, 218, 218, 128)     147584    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 108, 108, 128)    0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 106, 106, 128)     147584    
                                                           

In [17]:
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


In [26]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=50, epochs=10000)

RuntimeError: Physical devices cannot be modified after being initialized